# Data Cleaning

## Imports

In [1]:
import json
from pprint import pp
from functools import reduce
import re

### Read-in

In [2]:
with open('../Data/character_table.json', encoding="utf-8") as f:
    data = json.load(f)
    
with open('../Data/skill_table.json', encoding="utf-8") as f:
    skill_data = json.load(f)

## Transform

In [3]:
cleaned_data = {}
mapper = {"CASTER": "Caster",
          "MEDIC" : "Medic",
          "PIONEER" : "Vanguard",
          "SNIPER": "Sniper",
          "SPECIAL" : "Specialist",
          "SUPPORT" : "Supporter",
          "TANK" : "Defender",
          "WARRIOR" : "Guard",
          "TOKEN" : "Summoned Unit",
          "TRAP" : "Obstacle"}

replace_dict = {
    "atk" : "ATK",
    "def" : "DEF",
    "max hp" : "Max HP",
    "hp" : "HP",
    "block" : "Block",
    "range" : "Range"
}

pattern = re.compile(r'<[^>]*>')

In [4]:
def get_name(unit):
    return unit["name"]

In [5]:
def get_class(unit):
    return mapper[unit["profession"]]

In [6]:
def get_tags(unit):
    return unit["tagList"]

In [7]:
def get_trait(unit):
    if unit["description"]:
        return pattern.sub("",unit["description"])
    else:
        return None

In [8]:
def get_rarity(unit):
    return unit["rarity"] + 1

In [9]:
def get_talents(unit):
    talent_list = unit["talents"]
    try:
        return [{talent["name"] : talent["description"] for talent in talents["candidates"] if talent["requiredPotentialRank"] == 0} for talents in talent_list]
    except TypeError:
        return None

In [65]:
 def get_skills(unit):
    skill_list = []
    skills = [skill["skillId"] for skill in unit["skills"]]
    try:
        for i, skill in enumerate(skills):
            name = skill_data[skill]["levels"][0]["name"]
            bb = { row["key"] : row["value"] for row in skill_data[skill]["levels"][0]["blackboard"]}
            skill_text = skill_data[skill]["levels"][0]["description"].replace("-","")
    #         print(skill_text)
    #         print(bb)
            try:
                skill_text = reduce(lambda x,y: x.replace(y, replace_dict[y]), replace_dict, pattern.sub("",skill_text.lower().replace(":0%",":.0%").format(**bb)).capitalize())
            except KeyError:
                print(f"{get_name(unit)} Skill {i+1} was not successful")
                skill_text = skill_text
            unit_name = get_name(unit)
            skill_list.append({name : skill_text.replace(unit_name.lower(),unit_name)})
        return skill_list
    except KeyError:
        return None

In [66]:
cleaned_data = {}
for key, unit in data.items():
    cleaned_data[get_name(unit)] = {
        "rarity" : get_rarity(unit),
        "class" : get_class(unit),
        "tags" : get_tags(unit),
        "trait" : get_trait(unit),
        "talents" : get_talents(unit),
        "skills" : get_skills(unit),
        "internal_id" : key
    }

Courier Skill 2 was not successful
Mousse Skill 1 was not successful
Eyjafjalla Skill 1 was not successful
Saria Skill 3 was not successful


In [62]:
cleaned_data["Courier"]

{'rarity': 4,
 'class': 'Vanguard',
 'tags': ['DP-Recovery', 'Defense'],
 'trait': 'Blocks 2 enemies',
 'talents': [{'Karlan Patrol': 'DEF +16% when two or more enemies are blocked'}],
 'skills': [{'Charge β': 'Instantly gains 9.0 deployment points.'},
  {'Command - Defense': 'Immediately gains <@ba.vup>{blackd_s_2[once].cost}</> Deployment Points\nGradually obtains <@ba.vup>{blackd_s_2[period].trig_cnt}</> DP during the skill duration; DEF <@ba.vup>+{blackd_s_2[period].def:0%}</>'}],
 'internal_id': 'char_198_blackd'}

In [67]:
cleaned_data["Mousse"]

{'rarity': 4,
 'class': 'Guard',
 'tags': ['DPS'],
 'trait': 'Deals Arts Damage',
 'talents': [{'Combo': 'Has a 20% chance to attack twice in a row when attacking'}],
 'skills': [{'Scratch': "ATK <@ba.vup>+{atk:0%}</> for the next hit; The target's ATK <@ba.vup>{frncat_s_1[debuff].atk:0%}</> for <@ba.vup>{frncat_s_1[debuff].duration}</> seconds"},
  {'Fury': 'Atk and DEF +27%'}],
 'internal_id': 'char_185_frncat'}

In [68]:
cleaned_data["Eyjafjalla"]

{'rarity': 6,
 'class': 'Caster',
 'tags': ['DPS', 'Debuff'],
 'trait': 'Deals Arts Damage',
 'talents': [{'Pyrobreath': "All [Caster] Operators' ATK +14% when Eyjafjalla is deployed"},
  {'Wild Fire': 'Immediately obtains a small random number of Skill Points after deployment'}],
 'skills': [{'Duetto': 'ASPD <@ba.vup>+{amgoat_s_1[a].attack_speed}</>\nStarting from the second use of the skill, ATK <@ba.vup>+{amgoat_s_1[b].atk:0%}</>'},
  {'Ignition': 'Deals 240% of ATK arts damage in the next attack; deals half damage to enemies around the target and their res -10% for 6.0 seconds\ncan store 1.0 charge(s)'},
  {'Volcano': 'Atk +55%; Range increases; attack interval reduces significantly; attack becomes randomly shooting lava that can hit at most 3.0 enemies within Range'}],
 'internal_id': 'char_180_amgoat'}

In [69]:
cleaned_data["Saria"]

{'rarity': 6,
 'class': 'Defender',
 'tags': ['Defense', 'Healing', 'Support'],
 'trait': 'Can heal allies by using the skill',
 'talents': [{'Rhine Charged Suit': 'For every 20 seconds Saria is deployed, ATK +5% and DEF +4%, stacking up to 5 times'},
  {'Refreshment': "When Saria restores an ally's HP, she also restores 1 extra Skill Point to them"}],
 'skills': [{'First Aid': "Restores the HP of a nearby ally with less than 50% HP by 110% of the operator's ATK in the next attack\ncan store 1.0 charge(s)"},
  {'Medicine Dispensing': "Restores the HP of all allies within a certain Range by 80% of Saria's ATK"},
  {'Calcification': "Restores the HP of all nearby allies by <@ba.vup>{attack@heal_scale:0%}</> of Saria's ATK\nThe Arts damage taken by nearby enemies <@ba.vup>+{fake.b:0%}</> and their Movement Speed <@ba.vup>{demkni_s_3.move_speed:0%}</>"}],
 'internal_id': 'char_202_demkni'}

## Load

In [19]:
with open('../Data/cleaned_characters.json', 'w') as f:
    json.dump(cleaned_data, f)